In [1]:
import cv2
import numpy as np
import copy
import math
import time
import webbrowser
import os
import pyautogui as py
import glob
import time

In [2]:
def nothing(x):
    pass
image_x, image_y = 64,64
s=''
cap_region_x_begin=0.6  # start point/total width
cap_region_y_end=0.4  # start point/total width
threshold = 60  #  BINARY threshold
blurValue = 41  # GaussianBlur parameter
bgSubThreshold = 50
learningRate = 0
val=50.0
image_x, image_y = 64, 64

# variables
isBgCaptured = 0   # bool, whether the background captured
triggerSwitch = False

In [3]:
import pickle
clf = pickle.load(open('modelhand6.sav', 'rb'))

In [4]:
def ellipse_variance(contour):
    M=cv2.moments(contour)
    cX=int(M['m10']/M['m00'])
    cY=int(M['m01']/M['m00'])
    v=np.zeros((1,2,1))
    for cord in contour:
        x=np.array([[[cord[0][0]-cX],[cord[0][1]-cY]]])
        v=np.vstack((v,x))
#         print(v)
        # print(x)
#         print(cord)
#         print(cord[0][0])
#     print(v.shape)
    c_ellipse=np.zeros((2,2))
    for i in v:
        c_ellipse+=np.dot(i,np.transpose(i))
    c_ellipse/=contour.shape[0]
#     print(c_ellipse)
    c_ellipse_inv=np.linalg.inv(c_ellipse)
    d=np.zeros((1,1))
    for i in v:
        d=np.vstack((d,np.sqrt(np.dot(np.dot(np.transpose(i),c_ellipse_inv),i))))
    d=d[1:]
    mu=1/contour.shape[0]*np.sum(d)
    sigma=np.sqrt(1/contour.shape[0]*np.sum(np.square(d-mu)))
    return mu/sigma

In [5]:
t=[]
ma=0
isBgCaptured=0
cou=0
cv2.namedWindow("Trackbars")

cv2.createTrackbar("L - H", "Trackbars", 0, 179, nothing)
cv2.createTrackbar("L - S", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("L - V", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("U - H", "Trackbars", 179, 179, nothing)
cv2.createTrackbar("U - S", "Trackbars", 255, 255, nothing)
cv2.createTrackbar("U - V", "Trackbars", 50, 255, nothing)

def printThreshold(thr):
    print("! Changed threshold to "+str(thr))


def removeBG(frame):
    fgmask = bgModel.apply(frame,learningRate=learningRate)
    #kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    #res = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)

    kernel = np.ones((4, 4), np.uint8)
    fgmask = cv2.erode(fgmask, kernel, iterations=1)
    kernel = np.ones((1, 1), np.uint8)
    #fgmask = cv2.dilate(fgmask, kernel, iterations=6)
    res = cv2.bitwise_and(frame, frame, mask=fgmask)
    return res

camera = cv2.VideoCapture(0)
camera.set(10,200)
cv2.namedWindow('threshold')
cv2.createTrackbar('trh1', 'threshold', threshold, 100, printThreshold)
ret, fra = camera.read()
fra=cv2.cvtColor(fra,cv2.COLOR_BGR2GRAY)
for i in range(480):
    for j in range(640):
        fra[i][j]=1
while camera.isOpened():
    b=0
    k = cv2.waitKey(1)
    lower=np.array([90,90,90])
    upper=np.array([240,240,240])
    ret, frame = camera.read()
    alpha = 1
    beta = -70
    frame = cv2.addWeighted(frame, alpha, np.zeros(frame.shape, frame.dtype),0, beta)
    #print(frame.shape)
    blur = cv2.GaussianBlur(frame, (25, 25), 0)
    lower_blue = np.array([0, 12, 92])
    upper_blue = np.array([179, 255, 230])
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_blue, upper_blue)
    #frame = cv2.bitwise_and(frame, frame, mask=mask)
    t=[]
    #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    #threshold = cv2.getTrackbarPos('trh1', 'threshold')
    threshold = 60
    frame = cv2.bilateralFilter(frame, 5, 50, 100)  # smoothing filter
    frame = cv2.flip(frame, 1)  # flip the frame horizontally
    cv2.rectangle(frame, (int(cap_region_x_begin * frame.shape[1]), 0),
                 (frame.shape[1], int(cap_region_y_end * frame.shape[0])), (255, 0, 0), 2)
    cv2.imshow('original', frame)


    if isBgCaptured == 1:
        cou+=1
        # this part wont run until background captured
        #mas = cv2.inRange(frame, lower, upper)
        #img = cv2.bitwise_and(frame, frame, mask=mas)
        img = removeBG(frame)
        img = img[0:int(cap_region_y_end * frame.shape[0]),
                    int(cap_region_x_begin * frame.shape[1]):frame.shape[1]]  # clip the ROI

        # for i in range(img.shape[0]):
        #   for j in range(img.shape[1]):
        #     if img[i][j][0]<140 or img[i][j][0]>180:
        #       img[i][j][0]=0
        #       img[i][j][1]=0
        #       img[i][j][2]=0
#         l_h = cv2.getTrackbarPos("L - H", "Trackbars")
#         l_s = cv2.getTrackbarPos("L - S", "Trackbars")
#         l_v = cv2.getTrackbarPos("L - V", "Trackbars")
#         u_h = cv2.getTrackbarPos("U - H", "Trackbars")
#         u_s = cv2.getTrackbarPos("U - S", "Trackbars")
#         u_v = cv2.getTrackbarPos("U - V", "Trackbars")

        

#          lower_blue = np.array([0, 23, 92])
#          upper_blue = np.array([179, 255, 225])
        #hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        #mask = cv2.inRange(hsv, lower_blue, upper_blue)
        #cv2.imshow("m", hsv)
        cv2.imshow('mask', img)

        # convert the image into binary image
        
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (blurValue, blurValue), 0)
        cv2.imshow('blur', blur)
        #ret, thresh = cv2.threshold(blur, threshold, 255, cv2.THRESH_BINARY)
        ret3,thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        kernel = np.ones((5,5),np.uint8)
        opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
        kernel = np.ones((1,1),np.uint8)
        dilation = cv2.dilate(opening,kernel,iterations = 4)
        im2,contours,hierarchy=cv2.findContours(dilation,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        # print(len(contours))
#         for cnt in contours:
#             a = cv2.contourArea(cnt)
#             if a>ma:
#                 c=cnt;
        if len(contours)>0:
            req=max(contours,key=cv2.contourArea)
            # print(req.shape)
            #req=req[0]
            #req=c
            area=cv2.contourArea(req)
            #eccentricity
            dim_rect=cv2.minAreaRect(req)[1]
            para1=dim_rect[1]/dim_rect[0]
            #circularity
            perimeter=cv2.arcLength(req,True)
            para2=4*np.pi*area/(perimeter**2)
            #rectangularity
            para3=area/(dim_rect[1]*dim_rect[0])
            #convexity
            hull=cv2.convexHull(req)
            para4=cv2.arcLength(hull,True)/perimeter
            #solidity
            para5=area/cv2.contourArea(hull)
            #ellipse variance
            para6=ellipse_variance(req)
        #np.vstack((data,np.array([para1,para2,para3,para4,para5,para6,ord(label)])))
#         for cnt in contours:
#             a = cv2.contourArea(cnt)
#             if a>ma:
#                 c=cnt;
#         x,y,w,h = cv2.boundingRect(c)
#         imcrop = thresh[y:y+h, x:x+w]
#         cv2.imshow('ori', thresh)
        cv2.imshow('final', dilation)
#         cv2.imshow('final', imcrop)
#         img = cv2.resize(imcrop, (image_x, image_y), interpolation = cv2.INTER_AREA)
        if cou%50 == 0:
            time.sleep(2)
            g=[para1,para2,para3,para4,para5,para6]
            t.append(g)
            predicted = clf.predict(t)
            print(predicted)
            if predicted == 1:
                #time.sleep(2)
                py.FAILSAFE = False
                py.typewrite([' '])
                print(0)
            elif predicted == 2:
                py.FAILSAFE = False
                py.typewrite(['f'])
                print(2)
            elif predicted == 3:
                count=0
                cv2.waitKey(10)
                cv2.destroyAllWindows()
                while True:
                    count+=1
                    ret, fram = camera.read()
                    blur = cv2.GaussianBlur(fram, (25, 25), 0)
                    hsv = cv2.cvtColor(fram, cv2.COLOR_BGR2HSV)
                    mask = cv2.inRange(hsv, lower_blue, upper_blue)
                    fram = cv2.bitwise_and(fram, fram, mask=mask)
                    fram = cv2.bilateralFilter(fram, 5, 50, 100)  # smoothing filter
                    fram = cv2.flip(fram, 1)  # flip the frame horizontally
                    cv2.rectangle(fram, (int(cap_region_x_begin * fram.shape[1]), 0),
                                 (fram.shape[1], int(cap_region_y_end * fram.shape[0])), (255, 0, 0), 2)
                    cv2.imshow('original', fram)
                    if count>=200:
                        cv2.waitKey(10)
                        cv2.destroyAllWindows()
                        break
                    if count>=50:
                        imgi = removeBG(fram)
                        imgi = imgi[0:int(cap_region_y_end * fram.shape[0]),
                                    int(cap_region_x_begin * fram.shape[1]):fram.shape[1]]
                        cv2.imshow('mask', imgi)
        
                        gray = cv2.cvtColor(imgi, cv2.COLOR_BGR2GRAY)
                        blur = cv2.GaussianBlur(gray, (blurValue, blurValue), 0)
                        #cv2.imshow('blur', blur)
                        #ret, thresh = cv2.threshold(blur, threshold, 255, cv2.THRESH_BINARY)
                        ret3,thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
                        kernel = np.ones((5,5),np.uint8)
                        opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
                        kernel = np.ones((5,5),np.uint8)
                        dilation = cv2.dilate(opening,kernel,iterations = 1)
                        im2,contours,hierarchy=cv2.findContours(dilation,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
                        if len(contours)>0:
                            req=max(contours,key=cv2.contourArea)
                            x,y,w,h = cv2.boundingRect(req)
                            # print(req.shape)
                            #req=req[0]
                            #req=c
                            #area=cv2.contourArea(req)
                            area=w*h
                            print(area)
                            a=area/50000
                            if a<b:
                                py.hotkey('ctrl', 'f2')
                            elif a>b:
                                py.hotkey('ctrl', 'f3')
                            b=a
                            continue
                        else:
                            cv2.waitKey(10)
                            cv2.destroyAllWindows()
                            break
            elif predicted == 4:
                count=0
                #cv2.waitKey(10)
                #cv2.destroyAllWindows()
                while True:
                    count+=1
                    ret, fram = camera.read()
                    blur = cv2.GaussianBlur(fram, (25, 25), 0)
                    hsv = cv2.cvtColor(fram, cv2.COLOR_BGR2HSV)
                    mask = cv2.inRange(hsv, lower_blue, upper_blue)
                    fram = cv2.bitwise_and(fram, fram, mask=mask)
                    fram = cv2.bilateralFilter(fram, 5, 50, 100)  # smoothing filter
                    fram = cv2.flip(fram, 1)  # flip the frame horizontally
                    cv2.rectangle(fram, (int(cap_region_x_begin * fram.shape[1]), 0),
                                 (fram.shape[1], int(cap_region_y_end * fram.shape[0])), (255, 0, 0), 2)
                    cv2.imshow('original', fram)
                    if count>=200:
                        cv2.waitKey(10)
                        cv2.destroyAllWindows()
                        break
                    if count>=50:
                        imgi = removeBG(fram)
                        imgi = imgi[0:int(cap_region_y_end * fram.shape[0]),
                                    int(cap_region_x_begin * fram.shape[1]):fram.shape[1]]
                        #cv2.imshow('mask', imgi)
        
                        gray = cv2.cvtColor(imgi, cv2.COLOR_BGR2GRAY)
                        blur = cv2.GaussianBlur(gray, (blurValue, blurValue), 0)
                        #cv2.imshow('blur', blur)
                        #ret, thresh = cv2.threshold(blur, threshold, 255, cv2.THRESH_BINARY)
                        ret3,thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
                        kernel = np.ones((5,5),np.uint8)
                        opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
                        kernel = np.ones((5,5),np.uint8)
                        dilation = cv2.dilate(opening,kernel,iterations = 1)
                        im2,contours,hierarchy=cv2.findContours(dilation,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
                        if len(contours)>0:
                            req=max(contours,key=cv2.contourArea)
                            x,y,w,h = cv2.boundingRect(req)
                            # print(req.shape)
                            #req=req[0]
                            #req=c
                            #area=cv2.contourArea(req)
                            area=w*h
                            print(area)
                            a=area/50000
                            if a<b:
                                py.hotkey('ctrl', '2')
                            elif a>b:
                                py.hotkey('ctrl', '1')
                            b=a
                            continue
                        else:
                            cv2.waitKey(10)
                            cv2.destroyAllWindows()
                            break
                            
            elif predicted == 5:
                count=0
                cv2.waitKey(10)
                cv2.destroyAllWindows()
                while True:
                    count+=1
                    ret, fram = camera.read()
                    blur = cv2.GaussianBlur(fram, (25, 25), 0)
                    hsv = cv2.cvtColor(fram, cv2.COLOR_BGR2HSV)
                    mask = cv2.inRange(hsv, lower_blue, upper_blue)
                    frame = cv2.bitwise_and(fram, fram, mask=mask)
                    fram = cv2.bilateralFilter(fram, 5, 50, 100)  # smoothing filter
                    fram = cv2.flip(fram, 1)  # flip the frame horizontally
                    cv2.rectangle(fram, (int(cap_region_x_begin * fram.shape[1]), 0),
                                 (fram.shape[1], int(cap_region_y_end * fram.shape[0])), (255, 0, 0), 2)
                    #cv2.imshow('original', fram)
                    if count>=200:
                        cv2.waitKey(10)
                        cv2.destroyAllWindows()
                        break
                    if count>=50:
                        imgi = removeBG(fram)
                        imgi = imgi[0:int(cap_region_y_end * fram.shape[0]),
                                    int(cap_region_x_begin * fram.shape[1]):fram.shape[1]]
                        #cv2.imshow('mask', imgi)
                        gray = cv2.cvtColor(imgi, cv2.COLOR_BGR2GRAY)
                        blur = cv2.GaussianBlur(gray, (blurValue, blurValue), 0)
                        #cv2.imshow('blur', blur)
                        #ret, thresh = cv2.threshold(blur, threshold, 255, cv2.THRESH_BINARY)
                        ret3,thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
                        kernel = np.ones((5,5),np.uint8)
                        opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
                        kernel = np.ones((5,5),np.uint8)
                        dilation = cv2.dilate(opening,kernel,iterations = 1)
                        im2,contours,hierarchy=cv2.findContours(dilation,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
                        if len(contours)>0:
                            req=max(contours,key=cv2.contourArea)
                            x,y,w,h = cv2.boundingRect(req)
                            # print(req.shape)
                                #req=req[0]
                                #req=c
                                #area=cv2.contourArea(req)
                            area=w*h
                            print(area)
                            a=area/50000
#                             py.keyDown('ctrl')
                            if a>b:
                                py.hotkey('ctrl', 'right')
                            elif a<b:
                                py.hotkey('ctrl', 'left')
#                             py.keyUp('ctrl')
                            b=a
                            continue
                        else:
                            cv2.waitKey(10)
                            cv2.destroyAllWindows()
                            break
                                
            elif predicted==6:
                py.FAILSAFE = False
                #close window
                time.sleep(0.5)
                py.hotkey('alt', 'f4')
                    
            elif predicted==7:
                count=0
                cv2.waitKey(10)
                cv2.destroyAllWindows()
                while True:
                          count+=1
                    ret, fram = camera.read()
                    blur = cv2.GaussianBlur(fram, (25, 25), 0)
                    hsv = cv2.cvtColor(fram, cv2.COLOR_BGR2HSV)
                    mask = cv2.inRange(hsv, lower_blue, upper_blue)
                    fram = cv2.bitwise_and(fram, fram, mask=mask)
                    fram = cv2.bilateralFilter(fram, 5, 50, 100)  # smoothing filter
                    fram = cv2.flip(fram, 1)  # flip the frame horizontally
                    cv2.rectangle(fram, (int(cap_region_x_begin * fram.shape[1]), 0),
                                 (fram.shape[1], int(cap_region_y_end * fram.shape[0])), (255, 0, 0), 2)
                        #cv2.imshow('original', fram)
                    if count>=200:
                        cv2.waitKey(10)
                        cv2.destroyAllWindows()
                        break
                    if count>=50:
                        imgi = removeBG(fram)
                        imgi = imgi[0:int(cap_region_y_end * fram.shape[0]),
                                    int(cap_region_x_begin * fram.shape[1]):fram.shape[1]]
                            #cv2.imshow('mask', imgi)

                        gray = cv2.cvtColor(imgi, cv2.COLOR_BGR2GRAY)
                        blur = cv2.GaussianBlur(gray, (blurValue, blurValue), 0)
                            #cv2.imshow('blur', blur)
                            #ret, thresh = cv2.threshold(blur, threshold, 255, cv2.THRESH_BINARY)
                        ret3,thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
                        kernel = np.ones((5,5),np.uint8)
                        opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
                        kernel = np.ones((5,5),np.uint8)
                        dilation = cv2.dilate(opening,kernel,iterations = 1)
                        im2,contours,hierarchy=cv2.findContours(dilation,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
                        if len(contours)>0:
                            req=max(contours,key=cv2.contourArea)
                            x,y,w,h = cv2.boundingRect(req)
                            # print(req.shape)
                                #req=req[0]
                                #req=c
                                #area=cv2.contourArea(req)
                            area=w*h
                            print(area)
                            a=area/50000
                            #py.keyDown('shift')
                            if a<b:
                                py.typewrite(['['])
                            elif a>b:
                                py.typewrite([']'])
                            #py.keyUp('shift')
                            b=a
                            continue
                        else:
                            cv2.waitKey(10)
                            cv2.destroyAllWindows()
                            break
                                
            elif predicted==8:
                py.FAILSAFE = False
                time.sleep(0.5)
                py.hotkey('ctrl', 'alt', 'm')
                #minimize
#                 webbrowser.open("https://www.youtube.com/watch?v=du7paNLQN9M")
#                 time.sleep(5)
                
                
    if k == 27:  # press ESC to exit
        break
    elif k == ord('b'):  # press 'b' to capture the background
        bgModel = cv2.createBackgroundSubtractorMOG2(0, bgSubThreshold)
        isBgCaptured = 1
        print( '!!!Background Captured!!!')
    elif k == ord('r'):  # press 'r' to reset the background
        bgModel = None
        triggerSwitch = False
        isBgCaptured = 0
        print ('!!!Reset BackGround!!!')
    elif k == ord('n'):
        triggerSwitch = True
print ('!!!Trigger On!!!')
camera.release()
cv2.destroyAllWindows()

!!!Background Captured!!!
[1]
0
[1]
0
[1]
0
[1]
0
[1]
0
[1]
0
[2]
2
[2]
2
[1]
0
[1]
0
[2]
2
[1]
0
[1]
0
[2]
2
[1]
0
[1]
0
[1]
0
[1]
0
[1]
0
[5]
45756
44880
42823
41327
40018
39292
39123
38556
47628
47880
48132
48323
48960
49152
49152
48960
48960
48705
48514
48070
48070
47817
46937
46314
45880
45880
45880
45695
45880
46128
46314
46314
46065
45880
45880
45880
46065
46065
46065
46065
46065
46250
46065
46065
45816
46065
45816
46065
46065
46065
46065
46065
45880
45695
45695
45695
45695
45510
45942
45942
45942
45942
45942
45942
46376
46563
46812
47000
47628
47628
45441
46065
46500
46250
46065
45816
45816
45816
45816
45816
45816
45816
45816
45816
45816
45816
46065
44954
45632
45201
45201
45384
45384
45567
45567
45567
45567
45384
45567
45567
45567
45567
45567
45567
45567
45384
45567
45567
45567
45567
45567
45567
45567
45567
45567
45567
45567
45384
45567
45567
45567
45567
45567
45567
45567
45567
45567
45567
45567
45567
45567
45567
45567
45567
45567
45567
45384
45567
45567
45567
45567
45384
4538

12470
[1]
0
[1]
0
[1]
0
[1]
0
[1]
0
[5]
45567
45567
45567
45567
45567
45816
45567
45567
45567
45816
45567
45567
45567
45567
45816
45816
45567
45816
45567
45816
45567
45816
45816
45816
45816
45816
45816
45816
45816
45567
45567
45816
45816
45816
45816
45567
45816
45816
45816
45567
45816
45567
45816
45384
45384
45384
45384
45384
45384
45384
45384
45384
45384
45384
45384
45384
45384
45384
45384
45384
45384
45816
45816
45567
45816
45567
45567
45567
45567
45567
45567
45567
45567
45567
45384
45384
45632
45632
45632
45632
45816
45816
45567
45384
45384
45567
45384
45567
45816
45816
45816
45816
45816
45816
45816
45816
45816
45816
45816
45816
45567
45567
45384
45384
45567
45816
45384
45384
45384
45567
45384
45384
45384
45384
45384
45384
45384
45384
45384
45567
45567
45567
45567
45567
45567
45567
45567
45567
45567
45567
45567
45567
45567
45567
45567
45567
45567
45567
45567
45567
45567
45567
45567
45567
45567
45567
45567
45567
45567
45567
[5]
47311
47752
47752
47752
48195
48195
48006
46435
46250
46

46937
47124
47124
47124
47124
46500
47124
47376
47124
47124
[3]
46686
47564
46937
46937
47376
47564
47817
47564
46250
47817
46937
47817
48006
47564
48006
47376
48006
48006
47817
48450
47564
48450
47817
48260
48450
47817
48450
48006
47817
47817
47817
48006
48450
47817
47817
48006
48006
48450
47376
47124
48006
47564
47124
47564
46500
47124
46250
47124
46937
46937
46686
46250
47124
47564
46686
46686
46435
46872
47124
46686
46250
46250
46686
47564
47564
46686
46686
47124
48006
46937
47124
46937
48006
47817
47124
48006
47564
47817
47376
47124
46686
47124
47817
47124
46937
46686
47124
46937
47124
46937
47124
47124
47124
46065
46686
47124
47124
46937
46500
46500
47124
47124
47124
46686
46686
46500
47124
47124
46686
46872
46686
46686
46686
46250
47564
46250
46500
46937
46500
47124
46250
47124
46686
46686
46937
46686
46937
46250
46250
46250
46686
46686
46500
47124
46686
46686
47124
46686
46435
47124
46686
46686
46500
46250
46435
46686
46686
46686
46250
46686
[3]
46250
46250
47376
46937
47124
47

47817
47376
48450
47817
47817
47564
47817
48006
[1]
0
[1]
0
[8]
[8]
!!!Trigger On!!!


In [14]:
import glob
t=[]
y_test=[]
l=[]
y_train=[]
ma=0
labels=['A','B','G','L','Y']

def ellipse_variance(contour):
    M=cv2.moments(contour)
    cX=int(M['m10']/M['m00'])
    cY=int(M['m01']/M['m00'])
    v=np.zeros((1,2,1))
    for cord in contour:
        x=np.array([[[cord[0][0]-cX],[cord[0][1]-cY]]])
        v=np.vstack((v,x))  
#         print(v)
        # print(x)
#         print(cord)
#         print(cord[0][0])
#     print(v.shape)
    c_ellipse=np.zeros((2,2))
    for i in v:
        c_ellipse+=np.dot(i,np.transpose(i))
    c_ellipse/=contour.shape[0]
#     print(c_ellipse)
    c_ellipse_inv=np.linalg.inv(c_ellipse)
    d=np.zeros((1,1))
    for i in v:
        d=np.vstack((d,np.sqrt(np.dot(np.dot(np.transpose(i),c_ellipse_inv),i))))
    d=d[1:]
    mu=1/contour.shape[0]*np.sum(d)
    sigma=np.sqrt(1/contour.shape[0]*np.sum(np.square(d-mu)))
    return mu/sigma
count=0
#data=np.zeros((1,7))
for label in labels:
    count+=1
    path = "training_set/"+label+"/"
    cou=0
    for filename in os.listdir(path):
        cou+=1
        #print(filename,'1')
        input_path = os.path.join(path, filename)
        img=cv2.imread(input_path,0)
        ret,thresh=cv2.threshold(img,10,255,cv2.THRESH_BINARY)
        im2,contours,hierarchy=cv2.findContours(img,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        # print(len(contours))
#         for cnt in contours:
#             a = cv2.contourArea(cnt)
#             if a>ma:
#                 c=cnt;
        req=max(contours,key=cv2.contourArea)
        # print(req.shape)
        #req=req[0]
        #req=c
        area=cv2.contourArea(req)
        #eccentricity
        dim_rect=cv2.minAreaRect(req)[1]
        para1=dim_rect[1]/dim_rect[0]
        #circularity
        perimeter=cv2.arcLength(req,True)
        para2=4*np.pi*area/(perimeter**2)
        #rectangularity
        para3=area/(dim_rect[1]*dim_rect[0])
        #convexity
        hull=cv2.convexHull(req)
        para4=cv2.arcLength(hull,True)/perimeter
        #solidity
        para5=area/cv2.contourArea(hull)
        #ellipse variance
        para6=ellipse_variance(req)
        #np.vstack((data,np.array([para1,para2,para3,para4,para5,para6,ord(label)])))
        g=[para1,para2,para3,para4,para5,para6]
        l.append(g)
        y_train.append(count)
        

In [4]:
print(predicted)

NameError: name 'predicted' is not defined

In [12]:
firefox http://www.youtube.com

SyntaxError: invalid syntax (<ipython-input-12-2aa7b33b5d73>, line 1)

In [14]:
import webbrowser
webbrowser.open("https://www.youtube.com/watch?v=du7paNLQN9M")

True